In [1]:
!pip install ultralytics # Puedes especificar una versión si lo deseas
!pip install torchmetrics # Puedes especificar una versión si lo deseas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [2]:
# ===================== CONFIGURACIÓN INICIAL =====================
import os
import sys
import time
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.auto import tqdm
from datetime import datetime
from PIL import Image
import cv2
import yaml
import json
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import (
    OneCycleLR,
    ReduceLROnPlateau,
    CosineAnnealingLR,
    CosineAnnealingWarmRestarts
)
from torchvision import transforms
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    precision_recall_curve,
    average_precision_score,
    roc_curve,
    roc_auc_score
)
from ultralytics import YOLO
import gc

import wandb
import shutil

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import wandb

# Iniciar Weights & Biases para monitoreo
apy_key = '30a7a34eb742653f47d9208966cbed1dc6f31e2d' # Replace with your actual API key
wandb.login(key=apy_key) # Log in using the API key
wandb.init(project="yolov11_person_detectionV2", name="training_run", config={"seed": 42})

# Rest of your code...
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Utilizando dispositivo: {device}")
print(f"PyTorch versión: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA versión: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Configurar semillas para reproducibilidad
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(42)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gonzalesfranzreinaldo (gonzalesfranzreinaldo-usfx) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Utilizando dispositivo: cuda
PyTorch versión: 2.6.0+cu124
CUDA disponible: True
CUDA versión: 12.4
GPU: Tesla T4
Memoria GPU total: 15.83 GB


In [6]:
# ===================== CONFIGURACIÓN DE DIRECTORIOS =====================
DRIVE_PATH = '/content/drive/MyDrive'
DATASET_PATH = f'{DRIVE_PATH}/dataset_people'
OUTPUT_PATH = f'{DRIVE_PATH}/Trabajo_IA-3_FGS/yolov11_training'
CHECKPOINTS_PATH = f'{OUTPUT_PATH}/checkpoints'
LOGS_PATH = f'{OUTPUT_PATH}/logs'
RESULTS_PATH = f'{OUTPUT_PATH}/results'

# Crear directorios de salida
for path in [OUTPUT_PATH, CHECKPOINTS_PATH, LOGS_PATH, RESULTS_PATH]:
    os.makedirs(path, exist_ok=True)
print(f"Estructura de directorios creada en: {OUTPUT_PATH}")

# Verificar acceso a Google Drive
if not os.path.exists(DRIVE_PATH):
    raise FileNotFoundError("Google Drive no está montado correctamente")

Estructura de directorios creada en: /content/drive/MyDrive/Trabajo_IA-3_FGS/yolov11_training


In [7]:
# ===================== VERIFICACIÓN Y ANÁLISIS DEL DATASET =====================
print("\n=== ANÁLISIS DEL DATASET ===")
print(f"Ruta del dataset: {DATASET_PATH}")

def count_files(directory, extension='.jpg'):
    return len([f for f in os.listdir(directory) if f.endswith(extension)])

# Verificar estructura del dataset
images_train = f"{DATASET_PATH}/images/train"
images_val = f"{DATASET_PATH}/images/val"
images_test = f"{DATASET_PATH}/images/test"
labels_train = f"{DATASET_PATH}/labels/train"
labels_val = f"{DATASET_PATH}/labels/val"
labels_test = f"{DATASET_PATH}/labels/test"

# Contar imágenes y etiquetas
train_images_count = count_files(images_train, '.jpg')
val_images_count = count_files(images_val, '.jpg')
test_images_count = count_files(images_test, '.jpg')
train_labels_count = count_files(labels_train, '.txt')
val_labels_count = count_files(labels_val, '.txt')
test_labels_count = count_files(labels_test, '.txt')

# Verificación de integridad
print(f"Imágenes de entrenamiento: {train_images_count} (etiquetas: {train_labels_count})")
print(f"Imágenes de validación: {val_images_count} (etiquetas: {val_labels_count})")
print(f"Imágenes de prueba: {test_images_count} (etiquetas: {test_labels_count})")
if train_images_count != train_labels_count or val_images_count != val_labels_count or test_images_count != test_labels_count:
    raise ValueError("Mismatch entre imágenes y etiquetas")

# Analizar etiquetas
def analyze_labels(labels_dir, sample_size=100):
    label_files = [os.path.join(labels_dir, f) for f in os.listdir(labels_dir) if f.endswith('.txt')]
    if not label_files:
        return None
    sample_size = min(sample_size, len(label_files))
    selected_files = np.random.choice(label_files, sample_size, replace=False)
    boxes_per_image = []
    box_sizes = []
    for file_path in selected_files:
        with open(file_path, 'r') as f:
            lines = f.readlines()
            boxes_per_image.append(len(lines))
            for line in lines:
                values = line.strip().split()
                if len(values) == 5:
                    w, h = float(values[3]), float(values[4])
                    box_sizes.append(w * h)
    return {
        'avg_boxes_per_image': np.mean(boxes_per_image),
        'median_boxes_per_image': np.median(boxes_per_image),
        'max_boxes_per_image': np.max(boxes_per_image),
        'min_boxes_per_image': np.min(boxes_per_image),
        'avg_box_size': np.mean(box_sizes),
        'min_box_size': np.min(box_sizes),
        'max_box_size': np.max(box_sizes)
    }

print("\n=== ANÁLISIS DE ETIQUETAS ===")
train_stats = analyze_labels(labels_train)
if train_stats:
    print("Estadísticas del conjunto de entrenamiento:")
    print(f"- Promedio de cajas por imagen: {train_stats['avg_boxes_per_image']:.2f}")
    print(f"- Mediana de cajas por imagen: {train_stats['median_boxes_per_image']:.2f}")
    print(f"- Rango de cajas por imagen: {train_stats['min_boxes_per_image']} - {train_stats['max_boxes_per_image']}")
    print(f"- Tamaño promedio de caja: {train_stats['avg_box_size']:.4f}")
    print(f"- Rango de tamaños de caja: {train_stats['min_box_size']:.4f} - {train_stats['max_box_size']:.4f}")

# Verificar archivo data.yaml
yaml_path = f"{DATASET_PATH}/data.yaml"
if os.path.exists(yaml_path):
    with open(yaml_path, 'r') as f:
        data_yaml = yaml.safe_load(f)
    print("\nContenido del archivo data.yaml:")
    print(data_yaml)
else:
    print("\nCreando archivo data.yaml...")
    data_yaml = {
        'train': './images/train',
        'val': './images/val',
        'test': './images/test',
        'nc': 1,
        'names': ['persona'],
        'transforms': {'normalize': {'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}}
    }
    with open(yaml_path, 'w') as f:
        yaml.dump(data_yaml, f)
    print("Archivo data.yaml creado con éxito.")


=== ANÁLISIS DEL DATASET ===
Ruta del dataset: /content/drive/MyDrive/dataset_people
Imágenes de entrenamiento: 10200 (etiquetas: 10200)
Imágenes de validación: 2500 (etiquetas: 2500)
Imágenes de prueba: 1200 (etiquetas: 1200)

=== ANÁLISIS DE ETIQUETAS ===
Estadísticas del conjunto de entrenamiento:
- Promedio de cajas por imagen: 2.47
- Mediana de cajas por imagen: 2.00
- Rango de cajas por imagen: 1 - 8
- Tamaño promedio de caja: 0.0734
- Rango de tamaños de caja: 0.0084 - 0.2655

Contenido del archivo data.yaml:
{'names': ['persona'], 'nc': 1, 'test': './images/test', 'train': './images/train', 'val': './images/val'}


In [ ]:
# ===================== VISUALIZACIÓN DE MUESTRAS DEL DATASET =====================
def visualize_sample_images(images_dir, labels_dir, num_samples=12):
    image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.jpg')])
    if not image_files:
        print(f"No se encontraron imágenes en {images_dir}")
        return
    indices = np.random.choice(len(image_files), min(num_samples, len(image_files)), replace=False)
    fig, axes = plt.subplots(3, 4, figsize=(20, 15))
    axes = axes.flatten()
    for i, idx in enumerate(indices):
        img_name = image_files[idx]
        img_path = os.path.join(images_dir, img_name)
        label_path = os.path.join(labels_dir, img_name.replace('.jpg', '.txt'))
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    values = line.strip().split()
                    if len(values) == 5:
                        x_center = float(values[1]) * img.shape[1]
                        y_center = float(values[2]) * img.shape[0]
                        width = float(values[3]) * img.shape[1]
                        height = float(values[4]) * img.shape[0]
                        x1 = int(x_center - width / 2)
                        y1 = int(y_center - height / 2)
                        x2 = int(x_center + width / 2)
                        y2 = int(y_center + height / 2)
                        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(img, "Persona", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        axes[i].imshow(img)
        axes[i].set_title(f"Imagen: {img_name}")
        axes[i].axis('off')
    plt.tight_layout()
    plt.savefig(f"{RESULTS_PATH}/dataset_samples.png")
    plt.show()

print("\n=== VISUALIZACIÓN DE MUESTRAS ===")
visualize_sample_images(images_train, labels_train)

In [8]:
# ===================== PREPARACIÓN DEL ARCHIVO DE CONFIGURACIÓN =====================
def create_custom_yaml():
    custom_yaml_path = f"{OUTPUT_PATH}/custom_dataset.yaml"
    data = {
        'path': DATASET_PATH,
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'nc': 1,
        'names': ['persona'],
        'transforms': {'normalize': {'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}},
        'roboflow': {
            'workspace': 'proyecto-violencia-escolar',
            'project': 'deteccion-personas',
            'version': 1,
            'license': 'MIT',
            'url': 'https://universe.roboflow.com/proyecto-violencia-escolar/deteccion-personas/dataset/1'
        }
    }
    with open(custom_yaml_path, 'w') as f:
        yaml.dump(data, f, sort_keys=False)
    print(f"Archivo de configuración creado en: {custom_yaml_path}")
    return custom_yaml_path

custom_yaml_path = create_custom_yaml()

Archivo de configuración creado en: /content/drive/MyDrive/Trabajo_IA-3_FGS/yolov11_training/custom_dataset.yaml


In [9]:
# ===================== FUNCIONES DE MÉTRICAS Y EVALUACIÓN =====================
def compute_confusion_matrix(model, test_dir, test_labels_dir, conf_threshold=0.4, iou_threshold=0.5):
    """Calcula la matriz de confusión para detección de objetos."""
    model.eval()
    TP, FP, FN = 0, 0, 0
    image_files = [f for f in os.listdir(test_dir) if f.endswith('.jpg')]
    for img_file in tqdm(image_files, desc="Calculando matriz de confusión"):
        img_path = os.path.join(test_dir, img_file)
        label_path = os.path.join(test_labels_dir, img_file.replace('.jpg', '.txt'))
        gt_boxes = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    values = line.strip().split()
                    if len(values) == 5:
                        _, x, y, w, h = map(float, values)
                        gt_boxes.append([x, y, w, h])
        with torch.no_grad():
            results = model.predict(img_path, conf=conf_threshold, iou=iou_threshold, verbose=False)[0]
        pred_boxes = [box.xywhn[0].cpu().numpy() for box in results.boxes]
        matched_gt = set()
        for pred in pred_boxes:
            px, py, pw, ph = pred
            matched = False
            for i, gt in enumerate(gt_boxes):
                if i in matched_gt:
                    continue
                gx, gy, gw, gh = gt
                x_left = max(px - pw/2, gx - gw/2)
                y_top = max(py - ph/2, gy - gh/2)
                x_right = min(px + pw/2, gx + gw/2)
                y_bottom = min(py + ph/2, gy + gh/2)
                if x_right < x_left or y_bottom < y_top:
                    continue
                intersection = (x_right - x_left) * (y_bottom - y_top)
                p_area, g_area = pw * ph, gw * gh
                iou = intersection / (p_area + g_area - intersection)
                if iou >= iou_threshold:
                    matched_gt.add(i)
                    matched = True
                    break
            if matched:
                TP += 1
            else:
                FP += 1
        FN += len(gt_boxes) - len(matched_gt)
    cm = np.array([[TP, FP], [FN, 0]])  # TN no aplicable
    return cm

def plot_confusion_matrix(cm, save_path=None):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicciones')
    plt.ylabel('Ground Truth')
    plt.title('Matriz de Confusión')
    if save_path:
        plt.savefig(save_path)
    plt.show()

def calculate_precision_recall_thresholds(model, test_dir, test_labels_dir, thresholds=np.arange(0.05, 1.0, 0.05)):
    precisions, recalls = [], []
    total_gt = sum(len(open(os.path.join(test_labels_dir, f)).readlines())
                   for f in os.listdir(test_labels_dir) if f.endswith('.txt'))
    if total_gt == 0:
        print("No se encontraron anotaciones en el conjunto de prueba")
        return [], []
    for threshold in tqdm(thresholds, desc="Calculando PR curve"):
        correct_detections = 0
        total_predictions = 0
        for img_file in os.listdir(test_dir):
            if not img_file.endswith('.jpg'):
                continue
            img_path = os.path.join(test_dir, img_file)
            label_path = os.path.join(test_labels_dir, img_file.replace('.jpg', '.txt'))
            gt_boxes = []
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    for line in f.readlines():
                        values = line.strip().split()
                        if len(values) == 5:
                            _, x, y, w, h = map(float, values)
                            gt_boxes.append([x, y, w, h])
            with torch.no_grad():
                results = model.predict(img_path, conf=threshold, verbose=False)[0]
            pred_boxes = [box.xywhn[0].cpu().numpy() for box in results.boxes]
            total_predictions += len(pred_boxes)
            matched_gt = set()
            for pred in pred_boxes:
                px, py, pw, ph = pred
                for i, gt in enumerate(gt_boxes):
                    if i in matched_gt:
                        continue
                    gx, gy, gw, gh = gt
                    x_left = max(px - pw/2, gx - gw/2)
                    y_top = max(py - ph/2, gy - gh/2)
                    x_right = min(px + pw/2, gx + gw/2)
                    y_bottom = min(py + ph/2, gy + gh/2)
                    if x_right < x_left or y_bottom < y_top:
                        continue
                    intersection = (x_right - x_left) * (y_bottom - y_top)
                    p_area, g_area = pw * ph, gw * gh
                    iou = intersection / (p_area + g_area - intersection)
                    if iou >= 0.5:
                        matched_gt.add(i)
                        correct_detections += 1
                        break
        precision = correct_detections / total_predictions if total_predictions > 0 else 1.0
        recall = correct_detections / total_gt
        precisions.append(precision)
        recalls.append(recall)
    return thresholds, precisions, recalls

def calculate_roc_curve(model, test_dir, test_labels_dir, thresholds=np.arange(0.05, 1.0, 0.05)):
    tprs, fprs = [], []
    total_gt = sum(len(open(os.path.join(test_labels_dir, f)).readlines())
                   for f in os.listdir(test_labels_dir) if f.endswith('.txt'))
    for threshold in tqdm(thresholds, desc="Calculando ROC curve"):
        TP, FP, FN = 0, 0, 0
        for img_file in os.listdir(test_dir):
            if not img_file.endswith('.jpg'):
                continue
            img_path = os.path.join(test_dir, img_file)
            label_path = os.path.join(test_labels_dir, img_file.replace('.jpg', '.txt'))
            gt_boxes = []
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    for line in f.readlines():
                        values = line.strip().split()
                        if len(values) == 5:
                            _, x, y, w, h = map(float, values)
                            gt_boxes.append([x, y, w, h])
            with torch.no_grad():
                results = model.predict(img_path, conf=threshold, verbose=False)[0]
            pred_boxes = [box.xywhn[0].cpu().numpy() for box in results.boxes]
            matched_gt = set()
            for pred in pred_boxes:
                px, py, pw, ph = pred
                matched = False
                for i, gt in enumerate(gt_boxes):
                    if i in matched_gt:
                        continue
                    gx, gy, gw, gh = gt
                    x_left = max(px - pw/2, gx - gw/2)
                    y_top = max(py - ph/2, gy - gh/2)
                    x_right = min(px + pw/2, gx + gw/2)
                    y_bottom = min(py + ph/2, gy + gh/2)
                    if x_right < x_left or y_bottom < y_top:
                        continue
                    intersection = (x_right - x_left) * (y_bottom - y_top)
                    p_area, g_area = pw * ph, gw * gh
                    iou = intersection / (p_area + g_area - intersection)
                    if iou >= 0.5:
                        matched_gt.add(i)
                        matched = True
                        break
                if matched:
                    TP += 1
                else:
                    FP += 1
            FN += len(gt_boxes) - len(matched_gt)
        tpr = TP / (TP + FN) if (TP + FN) > 0 else 0
        fpr = FP / (FP + 0) if FP > 0 else 0
        tprs.append(tpr)
        fprs.append(fpr)
    auc = np.trapz(tprs, fprs)
    return thresholds, tprs, fprs, auc

def plot_precision_recall_curve(precisions, recalls, ap, save_path=None):
    plt.figure(figsize=(10, 6))
    plt.plot(recalls, precisions, 'b-', label=f'AP = {ap:.4f}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Curva de Precisión-Recall')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.grid(True)
    plt.legend()
    if save_path:
        plt.savefig(save_path)
    plt.show()

def plot_roc_curve(tprs, fprs, auc, save_path=None):
    plt.figure(figsize=(10, 6))
    plt.plot(fprs, tprs, 'b-', label=f'ROC (AUC = {auc:.4f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title('Curva ROC')
    plt.grid(True)
    plt.legend()
    if save_path:
        plt.savefig(save_path)
    plt.show()

def plot_learning_curves(results_df, save_path=None, phase='transfer'):
    plt.figure(figsize=(15, 10))
    box_loss = results_df['         train/box_loss'].values
    cls_loss = results_df['         train/cls_loss'].values
    dfl_loss = results_df['         train/dfl_loss'].values
    precision = results_df['       metrics/precision(B)'].values
    recall = results_df['          metrics/recall(B)'].values
    map50 = results_df['       metrics/mAP50(B)'].values
    map50_95 = results_df['    metrics/mAP50-95(B)'].values
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-6)

    plt.subplot(2, 2, 1)
    plt.plot(box_loss, label='Box Loss')
    plt.plot(cls_loss, label='Class Loss')
    plt.plot(dfl_loss, label='DFL Loss')
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.title(f'Evolución de las Pérdidas ({phase})')
    plt.legend()
    plt.grid(True)

    plt.subplot(2, 2, 2)
    plt.plot(map50, label='mAP@0.5')
    plt.plot(map50_95, label='mAP@0.5:0.95')
    plt.xlabel('Época')
    plt.ylabel('mAP')
    plt.title(f'Evolución del mAP ({phase})')
    plt.legend()
    plt.grid(True)

    plt.subplot(2, 2, 3)
    plt.plot(precision, label='Precisión')
    plt.plot(recall, label='Recall')
    plt.xlabel('Época')
    plt.ylabel('Valor')
    plt.title(f'Precisión y Recall ({phase})')
    plt.legend()
    plt.grid(True)

    plt.subplot(2, 2, 4)
    plt.plot(f1_score, label='F1-Score')
    plt.xlabel('Época')
    plt.ylabel('F1-Score')
    plt.title(f'Evolución del F1-Score ({phase})')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
    plt.show()

def plot_model_performance(metrics, save_path=None):
    plt.figure(figsize=(10, 6))
    sns.barplot(x=['Precisión', 'Recall', 'F1-Score'], y=[metrics['precision'], metrics['recall'], metrics['f1']])
    plt.ylim(0, 1)
    plt.title('Métricas de Rendimiento del Modelo')
    plt.ylabel('Valor')
    for i, v in enumerate([metrics['precision'], metrics['recall'], metrics['f1']]):
        plt.text(i, v + 0.02, f'{v:.4f}', ha='center')
    if save_path:
        plt.savefig(save_path)
    plt.show()

In [10]:
# ===================== CONFIGURACIÓN GENERAL =====================
def get_optimal_batch_size(vram_limit=16):
    vram = torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0
    return 8 if vram < vram_limit else 16

CONFIG = {
    'model_type': 'yolo11n',
    'img_size': 640,
    # 'batch_size': get_optimal_batch_size(),
    'batch_size': 16,
    'num_workers': 4,
    'epochs_transfer': 30, # Épocas para transfer learning
    'epochs_finetune': 50, # Épocas para fine-tuning
    'patience': 8,  # Early stopping patience
    'classes': ['persona'],   # Sólo una clase: persona
    'pretrained_weights': 'yolo11n.pt',   # Pesos pre-entrenados
    'learning_rate_transfer': 0.001,
    'learning_rate_finetune': 0.0001,
    'weight_decay': 0.0005,
    'momentum': 0.937,
    'save_period': 5  # Guardar cada 5 épocas
}

# ENTRENAMIENTO CON FINE TUNIGN V2

In [ ]:

# ===================== INICIANDO ENTRENAMIENTO: FINE-TUNING ITERACIÓN 2 =====================
CONFIG = {
    'model_type': 'yolo11n',
    'img_size': 640,  # Aumentado para mejorar localización
    'batch_size': 8,  # Reducido para imgsz=896
    'epochs_finetune': 50,
    'learning_rate_finetune': 0.00005,  # Reducido para ajustes más finos
    'patience': 10,  # Más estricto
    'num_workers': 4,
    'momentum': 0.937,
    'weight_decay': 0.0001,
    'save_period': 10
}

print("\n=== INICIANDO ENTRENAMIENTO: FINE-TUNING ITERACIÓN 2 ===")
print(f"Configuración de entrenamiento:")
print(f"- Modelo: {CONFIG['model_type']}")
print(f"- Tamaño de imagen: {CONFIG['img_size']}x{CONFIG['img_size']}")
print(f"- Batch size: {CONFIG['batch_size']}")
print(f"- Épocas: {CONFIG['epochs_finetune']}")
print(f"- Learning rate: {CONFIG['learning_rate_finetune']}")

# Cargar modelo del fine-tuning anterior
finetune_model_path = f"{OUTPUT_PATH}/fine_tuning_final.pt"
ft_model = YOLO(finetune_model_path)

# Configurar el entrenamiento
finetune_results = ft_model.train(
    data=custom_yaml_path,
    epochs=CONFIG['epochs_finetune'],
    imgsz=CONFIG['img_size'],
    batch=CONFIG['batch_size'],
    patience=CONFIG['patience'],
    save=True,
    save_period=CONFIG['save_period'],
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=CONFIG['num_workers'],
    lr0=CONFIG['learning_rate_finetune'],
    lrf=CONFIG['learning_rate_finetune'] / 10,
    momentum=CONFIG['momentum'],
    weight_decay=CONFIG['weight_decay'],
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    box=10.0,  # Aumentado para priorizar localización
    dfl=1.0,  # Reducido para equilibrar
    hsv_h=0.02,
    hsv_s=0.6,  # Reducido para estabilidad
    hsv_v=0.5,
    degrees=10.0,
    translate=0.2,
    scale=0.5,  # Reducido para estabilidad
    mixup=0.1,  # Añadido para diversidad
    close_mosaic=15,  # Más tiempo sin mosaic
    project=OUTPUT_PATH,
    name="fine_tuning_iter2",
    exist_ok=True,
    pretrained=True,
    optimizer="AdamW",
    verbose=True,
    seed=42,
    deterministic=True,
    cos_lr=True,
    freeze=15  # Más capas congeladas
)


# Guardar resultados
finetune_iter2_model_path = f"{OUTPUT_PATH}/fine_tuning_iter2_final.pt"
ft_model.save(finetune_iter2_model_path)
print(f"Modelo de fine-tuning iteración 2 guardado en: {finetune_iter2_model_path}")



=== INICIANDO ENTRENAMIENTO: FINE-TUNING ITERACIÓN 2 ===
Configuración de entrenamiento:
- Modelo: yolo11n
- Tamaño de imagen: 640x640
- Batch size: 8
- Épocas: 50
- Learning rate: 5e-05
Ultralytics 8.3.137 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=10.0, cache=False, cfg=None, classes=None, close_mosaic=15, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/Trabajo_IA-3_FGS/yolov11_training/custom_dataset.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.0, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=15, half=False, hsv_h=0.02, hsv_s=0.6, hsv_v=0.5, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=5e-05, lrf=5e-06, mask_ratio=4, max_det=300, mixup=0.1, mode

train: Scanning /content/drive/.shortcut-targets-by-id/1EKy5MKJmoLNQxoFyuOzjyUbOAB4YbrrM/dataset_people/labels/train.cache... 10200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10200/10200 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.5±0.2 ms, read: 45.9±21.6 MB/s, size: 82.8 KB)


val: Scanning /content/drive/.shortcut-targets-by-id/1EKy5MKJmoLNQxoFyuOzjyUbOAB4YbrrM/dataset_people/labels/val.cache... 2500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2500/2500 [00:00<?, ?it/s]


...

       3/50     0.961G      1.262     0.5491     0.7151         33        640: 100%|██████████| 1275/1275 [04:30<00:00,  4.71it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 157/157 [00:32<00:00,  4.89it/s]


                   all       2500       6662      0.956      0.966      0.989      0.742

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       4/50     0.967G      1.254     0.5427     0.7136         34        640: 100%|██████████| 1275/1275 [04:26<00:00,  4.78it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 157/157 [00:30<00:00,  5.23it/s]


                   all       2500       6662      0.962      0.962      0.989      0.745

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       5/50     0.967G       1.25     0.5514     0.7188         35        640:   8%|▊         | 96/1275 [00:19<03:49,  5.13it/s]

In [ ]:
# ===================== FUNCIÓN PARA GRAFICAR CURVAS DE APRENDIZAJE =====================
def plot_learning_curves(results_df, save_path=None, phase='transfer'):
    """
    Grafica las curvas de aprendizaje para pérdidas, mAP, precisión, recall y F1-score.

    Args:
        results_df (pd.DataFrame): DataFrame con los resultados del entrenamiento.
        save_path (str, optional): Ruta para guardar el gráfico.
        phase (str): Fase del entrenamiento ('transfer' o 'fine-tuning').
    """
    plt.figure(figsize=(15, 10))

    # Usar column_mapping para obtener los nombres correctos de las columnas
    try:
        box_loss = results_df[column_mapping['train/box_loss']].values
        cls_loss = results_df[column_mapping['train/cls_loss']].values
        dfl_loss = results_df[column_mapping['train/dfl_loss']].values
        precision = results_df[column_mapping['metrics/precision(B)']].values
        recall = results_df[column_mapping['metrics/recall(B)']].values
        map50 = results_df[column_mapping['metrics/mAP50(B)']].values
        map50_95 = results_df[column_mapping['metrics/mAP50-95(B)']].values
        f1_score = 2 * (precision * recall) / (precision + recall + 1e-6)
    except KeyError as e:
        print(f"Error al acceder a las columnas en plot_learning_curves: {e}")
        print("Columnas disponibles:", results_df.columns.tolist())
        raise

    # Subgráfico 1: Pérdidas
    plt.subplot(2, 2, 1)
    plt.plot(box_loss, label='Box Loss')
    plt.plot(cls_loss, label='Class Loss')
    plt.plot(dfl_loss, label='DFL Loss')
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.title(f'Evolución de las Pérdidas ({phase})')
    plt.legend()
    plt.grid(True)

    # Subgráfico 2: mAP
    plt.subplot(2, 2, 2)
    plt.plot(map50, label='mAP@0.5')
    plt.plot(map50_95, label='mAP@0.5:0.95')
    plt.xlabel('Época')
    plt.ylabel('mAP')
    plt.title(f'Evolución del mAP ({phase})')
    plt.legend()
    plt.grid(True)

    # Subgráfico 3: Precisión y Recall
    plt.subplot(2, 2, 3)
    plt.plot(precision, label='Precisión')
    plt.plot(recall, label='Recall')
    plt.xlabel('Época')
    plt.ylabel('Valor')
    plt.title(f'Precisión y Recall ({phase})')
    plt.legend()
    plt.grid(True)

    # Subgráfico 4: F1-Score
    plt.subplot(2, 2, 4)
    plt.plot(f1_score, label='F1-Score')
    plt.xlabel('Época')
    plt.ylabel('F1-Score')
    plt.title(f'Evolución del F1-Score ({phase})')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
        print(f"Gráfico guardado en: {save_path}")
    plt.show()





# Cargar resultados
results_csv_path = f"{OUTPUT_PATH}/fine_tuning_iter2/results.csv"
if not os.path.exists(results_csv_path):
    raise FileNotFoundError(f"No se encontró results.csv en {results_csv_path}")
results_df = pd.read_csv(results_csv_path)

# Inspeccionar nombres de columnas
print("Columnas en results.csv:")
print(results_df.columns.tolist())

# Usar el mismo column_mapping
column_mapping = {
    'train/box_loss': 'train/box_loss',
    'train/cls_loss': 'train/cls_loss',
    'train/dfl_loss': 'train/dfl_loss',
    'metrics/precision(B)': 'metrics/precision(B)',
    'metrics/recall(B)': 'metrics/recall(B)',
    'metrics/mAP50(B)': 'metrics/mAP50(B)',
    'metrics/mAP50-95(B)': 'metrics/mAP50-95(B)'
}

# Verificar y mapear columnas
available_columns = results_df.columns.tolist()
for expected_col, mapped_col in column_mapping.items():
    if mapped_col not in available_columns:
        similar_cols = [col for col in available_columns if expected_col.split('/')[-1] in col]
        if similar_cols:
            column_mapping[expected_col] = similar_cols[0]
            print(f"Advertencia: Columna '{mapped_col}' no encontrada, usando '{similar_cols[0]}' en su lugar")
        else:
            raise KeyError(f"No se encontró columna para '{expected_col}' en results.csv")

# Extraer métricas
try:
    box_loss = results_df[column_mapping['train/box_loss']].values
    cls_loss = results_df[column_mapping['train/cls_loss']].values
    dfl_loss = results_df[column_mapping['train/dfl_loss']].values
    precision = results_df[column_mapping['metrics/precision(B)']].values
    recall = results_df[column_mapping['metrics/recall(B)']].values
    map50 = results_df[column_mapping['metrics/mAP50(B)']].values
    map50_95 = results_df[column_mapping['metrics/mAP50-95(B)']].values
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-6)
except KeyError as e:
    print(f"Error al extraer métricas: {e}")
    raise

# Imprimir métricas finales
print("\n=== RESULTADOS DEL FINE-TUNING ITERACIÓN 2 ===")
print(f"Loss final (box): {box_loss[-1]:.4f}")
print(f"Loss final (cls): {cls_loss[-1]:.4f}")
print(f"Loss final (dfl): {dfl_loss[-1]:.4f}")
print(f"Precisión: {precision[-1]:.4f}")
print(f"Recall: {recall[-1]:.4f}")
print(f"F1-Score: {f1_score[-1]:.4f}")
print(f"mAP@0.5: {map50[-1]:.4f}")
print(f"mAP@0.5:0.95: {map50_95[-1]:.4f}")

# Graficar resultados
plot_learning_curves(results_df, f"{RESULTS_PATH}/fine_tuning_iter2_results.png", phase="Fine-Tuning Iteración 2")

# Loggear métricas en W&B
wandb.log({
    "finetune_iter2_box_loss": box_loss[-1],
    "finetune_iter2_cls_loss": cls_loss[-1],
    "finetune_iter2_dfl_loss": dfl_loss[-1],
    "finetune_iter2_precision": precision[-1],
    "finetune_iter2_recall": recall[-1],
    "finetune_iter2_f1": f1_score[-1],
    "finetune_iter2_map50": map50[-1],
    "finetune_iter2_map50_95": map50_95[-1]
})

# Liberar memoria
torch.cuda.empty_cache()
gc.collect()

# EVALUACION DEL MODELO FINAL

In [ ]:
# ===================== EVALUACIÓN DEL MODELO FINAL =====================
print("\n=== EVALUACIÓN DEL MODELO FINAL ===")
val_results = ft_model.val(data=custom_yaml_path, split='test')
val_metrics = val_results.results_dict
f1_score = 2 * val_metrics['metrics/precision(B)'] * val_metrics['metrics/recall(B)'] / \
           (val_metrics['metrics/precision(B)'] + val_metrics['metrics/recall(B)'] + 1e-6)

print("Métricas de evaluación:")
print(f"- Precisión: {val_metrics['metrics/precision(B)']:.4f}")
print(f"- Recall: {val_metrics['metrics/recall(B)']:.4f}")
print(f"- F1-Score: {f1_score:.4f}")
print(f"- mAP@0.5: {val_metrics['metrics/mAP50(B)']:.4f}")
print(f"- mAP@0.5:0.95: {val_metrics['metrics/mAP50-95(B)']:.4f}")

# Gráfico de radar
metrics_names = ['Precisión', 'Recall', 'F1-Score', 'mAP@0.5', 'mAP@0.5:0.95']
metrics_values = [
    val_metrics['metrics/precision(B)'],
    val_metrics['metrics/recall(B)'],
    f1_score,
    val_metrics['metrics/mAP50(B)'],
    val_metrics['metrics/mAP50-95(B)']
]
plt.figure(figsize=(10, 8))
angles = np.linspace(0, 2*np.pi, len(metrics_names), endpoint=False).tolist()
angles += angles[:1]
metrics_values += metrics_values[:1]
ax = plt.subplot(111, polar=True)
ax.plot(angles, metrics_values, 'o-', linewidth=2, color='blue')
ax.fill(angles, metrics_values, alpha=0.25, color='blue')
ax.set_thetagrids(np.degrees(angles[:-1]), metrics_names)
ax.set_ylim(0, 1)
plt.title('Métricas de Rendimiento del Modelo Final', pad=20)
plt.savefig(f"{RESULTS_PATH}/model_metrics_radar.png", dpi=300)
wandb.log({"model_metrics_radar": wandb.Image(f"{RESULTS_PATH}/model_metrics_radar.png")})
plt.show()

# Matriz de confusión
cm = compute_confusion_matrix(ft_model, images_test, labels_test)
plot_confusion_matrix(cm, f"{RESULTS_PATH}/confusion_matrix.png")
TP, FP, FN = cm[0, 0], cm[0, 1], cm[1, 0]
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
tpr = recall
fpr = FP / (FP + cm[1, 1]) if (FP + cm[1, 1]) > 0 else 0  # Corregido para incluir TN
print(f"Especificidad: No aplicable en detección de objetos")
print(f"TPR: {tpr:.4f}")
print(f"FPR: {fpr:.4f}")

# Curvas PR y ROC
thresholds, precisions, recalls = calculate_precision_recall_thresholds(ft_model, images_test, labels_test)
ap = 0
for i in range(1, len(recalls)):
    ap += (recalls[i] - recalls[i-1]) * precisions[i]
plot_precision_recall_curve(precisions, recalls, ap, f"{RESULTS_PATH}/precision_recall_curve.png")
wandb.log({"precision_recall_curve": wandb.Image(f"{RESULTS_PATH}/precision_recall_curve.png")})

thresholds, tprs, fprs, auc = calculate_roc_curve(ft_model, images_test, labels_test)
plot_roc_curve(tprs, fprs, auc, f"{RESULTS_PATH}/roc_curve.png")
wandb.log({"roc_curve": wandb.Image(f"{RESULTS_PATH}/roc_curve.png")})

# Loggear métricas finales
wandb.log({
    "test_precision": val_metrics['metrics/precision(B)'],
    "test_recall": val_metrics['metrics/recall(B)'],
    "test_f1": f1_score,
    "test_map50": val_metrics['metrics/mAP50(B)'],
    "test_map50_95": val_metrics['metrics/mAP50-95(B)'],
    "test_ap": ap,
    "test_roc_auc": auc,
    "test_tpr": tpr,
    "test_fpr": fpr
})

# ===================== VISUALIZACIÓN DE DETECCIONES =====================
def visualize_predictions(model, test_dir, num_samples=6, conf_threshold=0.25):
    """
    Visualiza predicciones del modelo en imágenes de prueba.

    Args:
        model: Modelo YOLO entrenado.
        test_dir (str): Directorio con imágenes de prueba.
        num_samples (int): Número de imágenes a visualizar.
        conf_threshold (float): Umbral de confianza para detecciones.
    """
    try:
        image_files = sorted([f for f in os.listdir(test_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])
        if not image_files:
            print(f"No se encontraron imágenes en {test_dir}")
            return
        indices = np.random.choice(len(image_files), min(num_samples, len(image_files)), replace=False)
        num_cols = min(len(indices), 3)
        num_rows = (len(indices) + num_cols - 1) // num_cols
        fig, axes = plt.subplots(num_rows, num_cols, figsize=(5*num_cols, 5*num_rows))
        axes = np.array(axes).flatten() if num_cols > 1 or num_rows > 1 else [axes]

        for i, idx in enumerate(indices):
            img_name = image_files[idx]
            img_path = os.path.join(test_dir, img_name)
            with torch.no_grad():
                results = model.predict(img_path, conf=conf_threshold, verbose=False)[0]
            im_array = results.plot(boxes=True, labels=True, conf=True)
            axes[i].imshow(cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB))
            axes[i].set_title(f"Predicción: {img_name}", fontsize=10)
            axes[i].axis('off')

        # Ocultar ejes vacíos si hay menos imágenes que subgráficos
        for i in range(len(indices), len(axes)):
            axes[i].axis('off')

        plt.tight_layout()
        save_path = f"{RESULTS_PATH}/sample_predictions.png"
        plt.savefig(save_path, dpi=300)
        wandb.log({"sample_predictions": wandb.Image(save_path)})
        print(f"Visualizaciones guardadas en: {save_path}")
        plt.show()

    except Exception as e:
        print(f"Error al visualizar predicciones: {str(e)}")

print("\n=== VISUALIZACIÓN DE DETECCIONES ===")
visualize_predictions(ft_model, images_test)

In [ ]:
# ===================== EVALUACIÓN DEL MODELO FINAL =====================
print("\n=== EVALUACIÓN DEL MODELO FINAL ===")
val_results = ft_model.val(data=custom_yaml_path, split='test')
val_metrics = val_results.results_dict
f1_score = 2 * val_metrics['metrics/precision(B)'] * val_metrics['metrics/recall(B)'] / \
           (val_metrics['metrics/precision(B)'] + val_metrics['metrics/recall(B)'] + 1e-6)

print("Métricas de evaluación:")
print(f"- Precisión: {val_metrics['metrics/precision(B)']:.4f}")
print(f"- Recall: {val_metrics['metrics/recall(B)']:.4f}")
print(f"- F1-Score: {f1_score:.4f}")
print(f"- mAP@0.5: {val_metrics['metrics/mAP50(B)']:.4f}")
print(f"- mAP@0.5:0.95: {val_metrics['metrics/mAP50-95(B)']:.4f}")

# Gráfico de radar
metrics_names = ['Precisión', 'Recall', 'F1-Score', 'mAP@0.5', 'mAP@0.5:0.95']
metrics_values = [
    val_metrics['metrics/precision(B)'],
    val_metrics['metrics/recall(B)'],
    f1_score,
    val_metrics['metrics/mAP50(B)'],
    val_metrics['metrics/mAP50-95(B)']
]
plt.figure(figsize=(10, 8))
angles = np.linspace(0, 2*np.pi, len(metrics_names), endpoint=False).tolist()
angles += angles[:1]
metrics_values += metrics_values[:1]
ax = plt.subplot(111, polar=True)
ax.plot(angles, metrics_values, 'o-', linewidth=2)
ax.fill(angles, metrics_values, alpha=0.25)
ax.set_thetagrids(np.degrees(angles[:-1]), metrics_names)
ax.set_ylim(0, 1)
plt.title('Métricas de Rendimiento del Modelo Final')
plt.savefig(f"{RESULTS_PATH}/model_metrics_radar.png")
plt.show()

# Matriz de confusión
cm = compute_confusion_matrix(ft_model, images_test, labels_test)
plot_confusion_matrix(cm, f"{RESULTS_PATH}/confusion_matrix.png")
TP, FP, FN = cm[0, 0], cm[0, 1], cm[1, 0]
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
tpr = recall
fpr = FP / (FP + 0) if FP > 0 else 0
print(f"Especificidad: No aplicable en detección de objetos")
print(f"TPR: {tpr:.4f}")
print(f"FPR: {fpr:.4f}")

# Curvas PR y ROC
thresholds, precisions, recalls = calculate_precision_recall_thresholds(ft_model, images_test, labels_test)
ap = 0
for i in range(1, len(recalls)):
    ap += (recalls[i] - recalls[i-1]) * precisions[i]
plot_precision_recall_curve(precisions, recalls, ap, f"{RESULTS_PATH}/precision_recall_curve.png")

thresholds, tprs, fprs, auc = calculate_roc_curve(ft_model, images_test, labels_test)
plot_roc_curve(tprs, fprs, auc, f"{RESULTS_PATH}/roc_curve.png")

# Loggear métricas finales
wandb.log({
    "test_precision": val_metrics['metrics/precision(B)'],
    "test_recall": val_metrics['metrics/recall(B)'],
    "test_f1": f1_score,
    "test_map50": val_metrics['metrics/mAP50(B)'],
    "test_map50_95": val_metrics['metrics/mAP50-95(B)'],
    "test_ap": ap,
    "test_roc_auc": auc
})


In [ ]:
# ===================== VISUALIZACIÓN DE DETECCIONES =====================
def visualize_predictions(model, test_dir, num_samples=6, conf_threshold=0.25):
    image_files = sorted([f for f in os.listdir(test_dir) if f.endswith('.jpg')])
    if not image_files:
        print(f"No se encontraron imágenes en {test_dir}")
        return
    indices = np.random.choice(len(image_files), min(num_samples, len(image_files)), replace=False)
    fig, axes = plt.subplots(2, len(indices)//2, figsize=(20, 10))
    axes = axes.flatten()
    for i, idx in enumerate(indices):
        img_name = image_files[idx]
        img_path = os.path.join(test_dir, img_name)
        with torch.no_grad():
            results = model.predict(img_path, conf=conf_threshold, verbose=False)[0]
        im_array = results.plot()
        axes[i].imshow(cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB))
        axes[i].set_title(f"Predicción: {img_name}")
        axes[i].axis('off')
    plt.tight_layout()
    plt.savefig(f"{RESULTS_PATH}/sample_predictions.png")
    plt.show()

print("\n=== VISUALIZACIÓN DE DETECCIONES ===")
visualize_predictions(ft_model, images_test)

In [ ]:
# ===================== ANÁLISIS DE RENDIMIENTO POR TAMAÑO DE OBJETO =====================
def analyze_performance_by_size(model, test_dir, test_labels_dir):
    size_categories = {'small': (0, 0.05), 'medium': (0.05, 0.15), 'large': (0.15, 1.0)}
    results_by_size = {size: {'correct': 0, 'total': 0, 'false_positives': 0} for size in size_categories}
    image_files = [f for f in os.listdir(test_dir) if f.endswith('.jpg')]
    for img_file in tqdm(image_files, desc="Evaluando por tamaño"):
        img_path = os.path.join(test_dir, img_file)
        label_path = os.path.join(test_labels_dir, img_file.replace('.jpg', '.txt'))
        gt_boxes = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    values = line.strip().split()
                    if len(values) == 5:
                        _, x, y, w, h = map(float, values)
                        area = w * h
                        size_cat = next((cat for cat, (min_size, max_size) in size_categories.items()
                                        if min_size <= area < max_size), None)
                        if size_cat:
                            results_by_size[size_cat]['total'] += 1
                            gt_boxes.append({'box': [x, y, w, h], 'size': size_cat, 'matched': False})
        with torch.no_grad():
            results = model.predict(img_path, conf=0.4, iou=0.5, verbose=False)[0]
        pred_boxes = []
        for box in results.boxes:
            x, y, w, h = box.xywhn[0].cpu().numpy()
            conf = box.conf[0].cpu().numpy()
            pred_boxes.append([x, y, w, h, conf])
        for pred in pred_boxes:
            best_iou = 0
            best_match = None
            for i, gt in enumerate(gt_boxes):
                if gt['matched']:
                    continue
                px, py, pw, ph, _ = pred
                gx, gy, gw, gh = gt['box']
                x_left = max(px - pw/2, gx - gw/2)
                y_top = max(py - ph/2, gy - gh/2)
                x_right = min(px + pw/2, gx + gw/2)
                y_bottom = min(py + ph/2, gy + gh/2)
                if x_right < x_left or y_bottom < y_top:
                    continue
                intersection = (x_right - x_left) * (y_bottom - y_top)
                p_area, g_area = pw * ph, gw * gh
                iou = intersection / (p_area + g_area - intersection)
                if iou > best_iou and iou >= 0.5:
                    best_iou = iou
                    best_match = i
            if best_match is not None:
                gt_boxes[best_match]['matched'] = True
                size = gt_boxes[best_match]['size']
                results_by_size[size]['correct'] += 1
            else:
                area = pred[2] * pred[3]
                for cat, (min_size, max_size) in size_categories.items():
                    if min_size <= area < max_size:
                        results_by_size[cat]['false_positives'] += 1
                        break
    metrics_by_size = {}
    for size, counts in results_by_size.items():
        if counts['total'] > 0:
            precision = counts['correct'] / (counts['correct'] + counts['false_positives']) if (counts['correct'] + counts['false_positives']) > 0 else 0
            recall = counts['correct'] / counts['total'] if counts['total'] > 0 else 0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
            metrics_by_size[size] = {
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'total': counts['total'],
                'correct': counts['correct'],
                'false_positives': counts['false_positives']
            }
    return metrics_by_size

print("\n=== ANÁLISIS DE RENDIMIENTO POR TAMAÑO DE OBJETO ===")
size_metrics = analyze_performance_by_size(ft_model, images_test, labels_test)
for size, metrics in size_metrics.items():
    print(f"- {size.capitalize()}:")
    print(f"  * Total: {metrics['total']}")
    print(f"  * Correctos: {metrics['correct']}")
    print(f"  * Falsos Positivos: {metrics['false_positives']}")
    print(f"  * Precisión: {metrics['precision']:.4f}")
    print(f"  * Recall: {metrics['recall']:.4f}")
    print(f"  * F1-Score: {metrics['f1']:.4f}")

sizes = list(size_metrics.keys())
precisions = [size_metrics[s]['precision'] for s in sizes]
recalls = [size_metrics[s]['recall'] for s in sizes]
f1_scores = [size_metrics[s]['f1'] for s in sizes]
x = np.arange(len(sizes))
width = 0.25
plt.figure(figsize=(12, 6))
plt.bar(x - width, precisions, width, label='Precisión')
plt.bar(x, recalls, width, label='Recall')
plt.bar(x + width, f1_scores, width, label='F1-Score')
plt.xlabel('Tamaño del Objeto')
plt.ylabel('Valor')
plt.title('Rendimiento por Tamaño de Objeto')
plt.xticks(x, [s.capitalize() for s in sizes])
plt.legend()
plt.grid(True, alpha=0.3)
for i, v in enumerate(precisions):
    plt.text(i - width, v + 0.02, f'{v:.2f}', ha='center')
for i, v in enumerate(recalls):
    plt.text(i, v + 0.02, f'{v:.2f}', ha='center')
for i, v in enumerate(f1_scores):
    plt.text(i + width, v + 0.02, f'{v:.2f}', ha='center')
plt.tight_layout()
plt.savefig(f"{RESULTS_PATH}/performance_by_size.png")
plt.show()

# Loggear métricas por tamaño
for size, metrics in size_metrics.items():
    wandb.log({f"size_{size}_precision": metrics['precision'],
               f"size_{size}_recall": metrics['recall'],
               f"size_{size}_f1": metrics['f1']})

In [ ]:
# ===================== ANÁLISIS POR REGIÓN DE LA IMAGEN =====================
def analyze_performance_by_region(model, test_dir, test_labels_dir):
    regions = {
        'top-left': (0, 0, 1/3, 1/3), 'top-center': (1/3, 0, 2/3, 1/3), 'top-right': (2/3, 0, 1, 1/3),
        'middle-left': (0, 1/3, 1/3, 2/3), 'middle-center': (1/3, 1/3, 2/3, 2/3), 'middle-right': (2/3, 1/3, 1, 2/3),
        'bottom-left': (0, 2/3, 1/3, 1), 'bottom-center': (1/3, 2/3, 2/3, 1), 'bottom-right': (2/3, 2/3, 1, 1)
    }
    results_by_region = {region: {'correct': 0, 'total': 0, 'false_positives': 0} for region in regions}
    image_files = [f for f in os.listdir(test_dir) if f.endswith('.jpg')]
    for img_file in tqdm(image_files, desc="Evaluando por región"):
        img_path = os.path.join(test_dir, img_file)
        label_path = os.path.join(test_labels_dir, img_file.replace('.jpg', '.txt'))
        gt_boxes = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    values = line.strip().split()
                    if len(values) == 5:
                        _, x, y, w, h = map(float, values)
                        region_key = next((key for key, (x1, y1, x2, y2) in regions.items()
                                          if x1 <= x < x2 and y1 <= y < y2), None)
                        if region_key:
                            results_by_region[region_key]['total'] += 1
                            gt_boxes.append({'box': [x, y, w, h], 'region': region_key, 'matched': False})
        with torch.no_grad():
            results = model.predict(img_path, conf=0.4, iou=0.5, verbose=False)[0]
        pred_boxes = []
        for box in results.boxes:
            x, y, w, h = box.xywhn[0].cpu().numpy()
            conf = box.conf[0].cpu().numpy()
            pred_boxes.append([x, y, w, h, conf])
        for pred in pred_boxes:
            best_iou = 0
            best_match = None
            for i, gt in enumerate(gt_boxes):
                if gt['matched']:
                    continue
                px, py, pw, ph, _ = pred
                gx, gy, gw, gh = gt['box']
                x_left = max(px - pw/2, gx - gw/2)
                y_top = max(py - ph/2, gy - gh/2)
                x_right = min(px + pw/2, gx + gw/2)
                y_bottom = min(py + ph/2, gy + gh/2)
                if x_right < x_left or y_bottom < y_top:
                    continue
                intersection = (x_right - x_left) * (y_bottom - y_top)
                p_area, g_area = pw * ph, gw * gh
                iou = intersection / (p_area + g_area - intersection)
                if iou > best_iou and iou >= 0.5:
                    best_iou = iou
                    best_match = i
            if best_match is not None:
                gt_boxes[best_match]['matched'] = True
                region = gt_boxes[best_match]['region']
                results_by_region[region]['correct'] += 1
            else:
                px, py = pred[0], pred[1]
                for key, (x1, y1, x2, y2) in regions.items():
                    if x1 <= px < x2 and y1 <= py < y2:
                        results_by_region[key]['false_positives'] += 1
                        break
    metrics_by_region = {}
    for region, counts in results_by_region.items():
        if counts['total'] > 0 or counts['false_positives'] > 0:
            precision = counts['correct'] / (counts['correct'] + counts['false_positives']) if (counts['correct'] + counts['false_positives']) > 0 else 0
            recall = counts['correct'] / counts['total'] if counts['total'] > 0 else 0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
            metrics_by_region[region] = {
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'total': counts['total'],
                'correct': counts['correct'],
                'false_positives': counts['false_positives']
            }
    return metrics_by_region

print("\n=== ANÁLISIS POR REGIÓN DE LA IMAGEN ===")
region_metrics = analyze_performance_by_region(ft_model, images_test, labels_test)
plt.figure(figsize=(12, 10))
grid = (3, 3)
positions = {
    'top-left': (0, 0), 'top-center': (0, 1), 'top-right': (0, 2),
    'middle-left': (1, 0), 'middle-center': (1, 1), 'middle-right': (1, 2),
    'bottom-left': (2, 0), 'bottom-center': (2, 1), 'bottom-right': (2, 2)
}
max_objects = max([m['total'] for m in region_metrics.values()]) if region_metrics else 1
max_f1 = max([m['f1'] for m in region_metrics.values()]) if region_metrics else 1
f1_matrix = np.zeros(grid)
density_matrix = np.zeros(grid)
for region, metrics in region_metrics.items():
    row, col = positions[region]
    f1_matrix[row, col] = metrics['f1']
    density_matrix[row, col] = metrics['total'] / max_objects if max_objects > 0 else 0
plt.subplot(1, 2, 1)
sns.heatmap(f1_matrix, annot=True, fmt=".2f", cmap="YlGnBu", vmin=0, vmax=1)
plt.title('F1-Score por Región')
plt.xticks([])
plt.yticks([])
plt.subplot(1, 2, 2)
sns.heatmap(density_matrix, annot=True, fmt=".2f", cmap="YlOrRd", vmin=0, vmax=1)
plt.title('Densidad de Objetos por Región')
plt.xticks([])
plt.yticks([])
plt.tight_layout()
plt.savefig(f"{RESULTS_PATH}/region_analysis.png")
plt.show()

print("Métricas detalladas por región:")
for region, metrics in sorted(region_metrics.items()):
    print(f"- {region}:")
    print(f"  * Total objetos: {metrics['total']}")
    print(f"  * Detecciones correctas: {metrics['correct']}")
    print(f"  * Falsos positivos: {metrics['false_positives']}")
    print(f"  * Precisión: {metrics['precision']:.4f}")
    print(f"  * Recall: {metrics['recall']:.4f}")
    print(f"  * F1-Score: {metrics['f1']:.4f}")

# Loggear métricas por región
for region, metrics in region_metrics.items():
    wandb.log({f"region_{region}_precision": metrics['precision'],
               f"region_{region}_recall": metrics['recall'],
               f"region_{region}_f1": metrics['f1']})

In [ ]:
# ===================== ANÁLISIS DE VELOCIDAD DE INFERENCIA =====================
def benchmark_inference_speed(model, test_dir, batch_sizes=[1, 2, 4, 8], num_runs=10):
    image_files = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if f.endswith('.jpg')]
    if len(image_files) > 50:
        image_files = np.random.choice(image_files, 50, replace=False).tolist()
    results = {}
    for batch_size in batch_sizes:
        times = []
        batches = [image_files[i:i+batch_size] for i in range(0, len(image_files), batch_size)]
        with torch.no_grad():
            _ = model(batches[0], verbose=False)
        for _ in range(num_runs):
            for batch in batches:
                start_time = time.time()
                with torch.no_grad():
                    _ = model(batch, verbose=False)
                end_time = time.time()
                times.append((end_time - start_time) / len(batch))
        avg_time = np.mean(times)
        std_time = np.std(times)
        fps = 1.0 / avg_time
        results[batch_size] = {'avg_time': avg_time, 'std_time': std_time, 'fps': fps}
    return results

print("\n=== ANÁLISIS DE VELOCIDAD DE INFERENCIA ===")
speed_results = benchmark_inference_speed(ft_model, images_test)
for batch_size, metrics in speed_results.items():
    print(f"- Batch size {batch_size}:")
    print(f"  * Tiempo promedio por imagen: {metrics['avg_time']*1000:.2f} ms")
    print(f"  * FPS: {metrics['fps']:.2f}")

plt.figure(figsize=(12, 5))
batch_sizes = list(speed_results.keys())
avg_times = [metrics['avg_time']*1000 for metrics in speed_results.values()]
std_times = [metrics['std_time']*1000 for metrics in speed_results.values()]
fps_values = [metrics['fps'] for metrics in speed_results.values()]
plt.subplot(1, 2, 1)
plt.errorbar(batch_sizes, avg_times, yerr=std_times, fmt='o-', capsize=5)
plt.xlabel('Tamaño de Batch')
plt.ylabel('Tiempo de Inferencia (ms)')
plt.title('Tiempo de Inferencia por Imagen')
plt.grid(True, alpha=0.3)
plt.subplot(1, 2, 2)
plt.bar(batch_sizes, fps_values, color='skyblue')
plt.xlabel('Tamaño de Batch')
plt.ylabel('FPS')
plt.title('Frames Por Segundo')
plt.grid(True, alpha=0.3)
for i, fps in enumerate(fps_values):
    plt.text(batch_sizes[i], fps + 0.5, f'{fps:.2f}', ha='center')
plt.tight_layout()
plt.savefig(f"{RESULTS_PATH}/inference_speed.png")
plt.show()

# Loggear velocidad
wandb.log({f"speed_batch_{bs}_fps": metrics['fps'] for bs, metrics in speed_results.items()})


In [ ]:
# ===================== FUNCIÓN PARA GRAFICAR CURVAS DE APRENDIZAJE =====================
def plot_learning_curves(results_df, save_path=None, phase='transfer'):
    """
    Grafica las curvas de aprendizaje para pérdidas, mAP, precisión, recall y F1-score.

    Args:
        results_df (pd.DataFrame): DataFrame con los resultados del entrenamiento.
        save_path (str, optional): Ruta para guardar el gráfico.
        phase (str): Fase del entrenamiento ('transfer' o 'fine-tuning').
    """
    plt.figure(figsize=(15, 10))

    # Usar column_mapping para obtener los nombres correctos de las columnas
    try:
        # Use the corrected column mapping
        box_loss = results_df[column_mapping['train/box_loss']].values
        cls_loss = results_df[column_mapping['train/cls_loss']].values
        dfl_loss = results_df[column_mapping['train/dfl_loss']].values
        precision = results_df[column_mapping['metrics/precision(B)']].values
        recall = results_df[column_mapping['metrics/recall(B)']].values
        map50 = results_df[column_mapping['metrics/mAP50(B)']].values
        map50_95 = results_df[column_mapping['metrics/mAP50-95(B)']].values
        f1_score = 2 * (precision * recall) / (precision + recall + 1e-6)
    except KeyError as e:
        print(f"Error al acceder a las columnas en plot_learning_curves: {e}")
        print("Columnas disponibles:", results_df.columns.tolist())
        # Re-raise the exception after printing diagnostic info
        raise

    # Subgráfico 1: Pérdidas
    plt.subplot(2, 2, 1)
    plt.plot(box_loss, label='Box Loss')
    plt.plot(cls_loss, label='Class Loss')
    plt.plot(dfl_loss, label='DFL Loss')
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.title(f'Evolución de las Pérdidas ({phase})')
    plt.legend()
    plt.grid(True)

    # Subgráfico 2: mAP
    plt.subplot(2, 2, 2)
    plt.plot(map50, label='mAP@0.5')
    plt.plot(map50_95, label='mAP@0.5:0.95')
    plt.xlabel('Época')
    plt.ylabel('mAP')
    plt.title(f'Evolución del mAP ({phase})')
    plt.legend()
    plt.grid(True)

    # Subgráfico 3: Precisión y Recall
    plt.subplot(2, 2, 3)
    plt.plot(precision, label='Precisión')
    plt.plot(recall, label='Recall')
    plt.xlabel('Época')
    plt.ylabel('Valor')
    plt.title(f'Precisión y Recall ({phase})')
    plt.legend()
    plt.grid(True)

    # Subgráfico 4: F1-Score
    plt.subplot(2, 2, 4)
    plt.plot(f1_score, label='F1-Score')
    plt.xlabel('Época')
    plt.ylabel('F1-Score')
    plt.title(f'Evolución del F1-Score ({phase})')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
        print(f"Gráfico guardado en: {save_path}")
    plt.show()

# ===================== CARGA Y ANÁLISIS DE RESULTADOS =====================
print("\n=== CARGANDO RESULTADOS DEL TRANSFER LEARNING ===")

# Cargar resultados
results_csv_path_transfer = f"{OUTPUT_PATH}/transfer_learning/results.csv"
results_csv_path_finetune = f"{OUTPUT_PATH}/fine_tuning/results.csv"

if not os.path.exists(results_csv_path_transfer):
    raise FileNotFoundError(f"No se encontró results.csv en {results_csv_path_transfer}")
if not os.path.exists(results_csv_path_finetune):
    raise FileNotFoundError(f"No se encontró results.csv en {results_csv_path_finetune}")

# Leer el archivo CSV y limpiar nombres de columnas
transfer_df = pd.read_csv(results_csv_path_transfer)
transfer_df.columns = transfer_df.columns.str.strip() # Strip whitespace from column names

finetune_df = pd.read_csv(results_csv_path_finetune)
finetune_df.columns = finetune_df.columns.str.strip() # Strip whitespace from column names


# Inspeccionar nombres de columnas
print("Columnas en results.csv (Transfer Learning):")
print(transfer_df.columns.tolist())
print("Columnas en results.csv (Fine-Tuning):")
print(finetune_df.columns.tolist())


# Definir nombres de columnas esperados (basado en la salida proporcionada, stripped)
# We need to make sure these keys match the stripped column names
column_mapping = {
    'train/box_loss': 'train/box_loss',
    'train/cls_loss': 'train/cls_loss',
    'train/dfl_loss': 'train/dfl_loss',
    'metrics/precision(B)': 'metrics/precision(B)',
    'metrics/recall(B)': 'metrics/recall(B)',
    'metrics/mAP50(B)': 'metrics/mAP50(B)',
    'metrics/mAP50-95(B)': 'metrics/mAP50-95(B)'
}

# Verify and map columns (optional, as stripping should make names consistent)
# You can keep this verification or remove it if confident in stripping
available_columns_transfer = transfer_df.columns.tolist()
available_columns_finetune = finetune_df.columns.tolist()

for expected_col, mapped_col in list(column_mapping.items()): # Use list() to allow modification
    if mapped_col not in available_columns_transfer or mapped_col not in available_columns_finetune:
         # Attempt a more flexible match if stripping isn't enough
         # Find a column that contains the essential part of the key
        key_part = expected_col.split('/')[-1].replace('(B)', '').strip()
        similar_cols_transfer = [col for col in available_columns_transfer if key_part in col]
        similar_cols_finetune = [col for col in available_columns_finetune if key_part in col]

        if similar_cols_transfer and similar_cols_finetune and similar_cols_transfer[0] == similar_cols_finetune[0]:
            column_mapping[expected_col] = similar_cols_transfer[0]
            print(f"Advertencia: Columna '{mapped_col}' no encontrada exactamente, usando '{similar_cols_transfer[0]}' en su lugar.")
        else:
            # If flexible matching fails, check original columns before stripping for more context
            original_transfer_columns = pd.read_csv(results_csv_path_transfer).columns.tolist()
            original_finetune_columns = pd.read_csv(results_csv_path_finetune).columns.tolist()
            print("\n--- Debug Info ---")
            print("Original Transfer Columns:", original_transfer_columns)
            print("Original Finetune Columns:", original_finetune_columns)
            print("Available Stripped Transfer Columns:", available_columns_transfer)
            print("Available Stripped Finetune Columns:", available_columns_finetune)
            print(f"Attempted to match key part: '{key_part}'")
            print(f"Similar columns found in Transfer: {similar_cols_transfer}")
            print(f"Similar columns found in Finetune: {similar_cols_finetune}")
            print("------------------\n")

            raise KeyError(f"No se encontró una columna que coincida con '{expected_col}' en ambos archivos results.csv después de intentar limpiar nombres. Verifique los nombres de las columnas.")

# Extract metrics using the mapped names
try:
    metrics_to_plot = {
        'loss': {
            'transfer': transfer_df[column_mapping['train/box_loss']] + transfer_df[column_mapping['train/cls_loss']] + transfer_df[column_mapping['train/dfl_loss']],
            'finetune': finetune_df[column_mapping['train/box_loss']] + finetune_df[column_mapping['train/cls_loss']] + finetune_df[column_mapping['train/dfl_loss']]
        },
        'map50': {
            'transfer': transfer_df[column_mapping['metrics/mAP50(B)']],
            'finetune': finetune_df[column_mapping['metrics/mAP50(B)']]
        },
        'map50-95': {
            'transfer': transfer_df[column_mapping['metrics/mAP50-95(B)']],
            'finetune': finetune_df[column_mapping['metrics/mAP50-95(B)']]
        },
        'precision': {
            'transfer': transfer_df[column_mapping['metrics/precision(B)']],
            'finetune': finetune_df[column_mapping['metrics/precision(B)']]
        },
        'recall': {
            'transfer': transfer_df[column_mapping['metrics/recall(B)']],
            'finetune': finetune_df[column_mapping['metrics/recall(B)']]
        }
    }
except KeyError as e:
    print(f"Error final al acceder a las columnas después del mapeo: {e}")
    # Print the final mapping to see what was used
    print("Final Column Mapping Used:", column_mapping)
    raise

# Plotting the curves
plt.figure(figsize=(15, 10))
plt.subplot(2, 2, 1)
plt.plot(range(1, len(metrics_to_plot['loss']['transfer']) + 1), metrics_to_plot['loss']['transfer'], 'b-', label='Transfer Learning')
plt.plot(np.arange(len(metrics_to_plot['loss']['transfer']) + 1, len(metrics_to_plot['loss']['transfer']) + len(metrics_to_plot['loss']['finetune']) + 1),
         metrics_to_plot['loss']['finetune'], 'r-', label='Fine-Tuning')
plt.axvline(x=len(metrics_to_plot['loss']['transfer']), color='k', linestyle='--')
plt.xlabel('Época')
plt.ylabel('Pérdida Total')
plt.title('Evolución de la Pérdida')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
plt.plot(range(1, len(metrics_to_plot['map50']['transfer']) + 1), metrics_to_plot['map50']['transfer'], 'b-', label='Transfer Learning')
plt.plot(np.arange(len(metrics_to_plot['map50']['transfer']) + 1, len(metrics_to_plot['map50']['transfer']) + len(metrics_to_plot['map50']['finetune']) + 1),
         metrics_to_plot['map50']['finetune'], 'r-', label='Fine-Tuning')
plt.axvline(x=len(metrics_to_plot['map50']['transfer']), color='k', linestyle='--')
plt.xlabel('Época')
plt.ylabel('mAP@0.5')
plt.title('Evolución del mAP@0.5')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
plt.plot(range(1, len(metrics_to_plot['map50-95']['transfer']) + 1), metrics_to_plot['map50-95']['transfer'], 'b-', label='Transfer Learning')
plt.plot(np.arange(len(metrics_to_plot['map50-95']['transfer']) + 1, len(metrics_to_plot['map50-95']['transfer']) + len(metrics_to_plot['map50-95']['finetune']) + 1),
         metrics_to_plot['map50-95']['finetune'], 'r-', label='Fine-Tuning')
plt.axvline(x=len(metrics_to_plot['map50-95']['transfer']), color='k', linestyle='--')
plt.xlabel('Época')
plt.ylabel('mAP@0.5:0.95')
plt.title('Evolución del mAP@0.5:0.95')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 4)
plt.plot(range(1, len(metrics_to_plot['precision']['transfer']) + 1), metrics_to_plot['precision']['transfer'], 'b-', label='Precisión (Transfer)')
plt.plot(np.arange(len(metrics_to_plot['precision']['transfer']) + 1, len(metrics_to_plot['precision']['transfer']) + len(metrics_to_plot['precision']['finetune']) + 1),
         metrics_to_plot['precision']['finetune'], 'r-', label='Precisión (Fine-Tune)')
plt.plot(range(1, len(metrics_to_plot['recall']['transfer']) + 1), metrics_to_plot['recall']['transfer'], 'b--', label='Recall (Transfer)')
plt.plot(np.arange(len(metrics_to_plot['recall']['transfer']) + 1, len(metrics_to_plot['recall']['transfer']) + len(metrics_to_plot['recall']['finetune']) + 1),
         metrics_to_plot['recall']['finetune'], 'r--', label='Recall (Fine-Tune)')
plt.axvline(x=len(metrics_to_plot['precision']['transfer']), color='k', linestyle='--')
plt.xlabel('Época')
plt.ylabel('Valor')
plt.title('Evolución de Precisión y Recall')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{RESULTS_PATH}/learning_curves_combined.png")
plt.show()

# Loggear curvas de aprendizaje
# Ensure the epoch ranges match the data
epochs_transfer = len(metrics_to_plot['loss']['transfer'])
epochs_finetune = len(metrics_to_plot['loss']['finetune'])

wandb.log({"combined_loss": wandb.plot.line_series(
    xs=list(range(1, epochs_transfer + epochs_finetune + 1)),
    ys=[list(metrics_to_plot['loss']['transfer']) + list(metrics_to_plot['loss']['finetune'])],
    keys=["Pérdida"],
    title="Evolución de la Pérdida"
)})

wandb.log({"combined_map50": wandb.plot.line_series(
    xs=list(range(1, epochs_transfer + epochs_finetune + 1)),
    ys=[list(metrics_to_plot['map50']['transfer']) + list(metrics_to_plot['map50']['finetune'])],
    keys=["mAP@0.5"],
    title="Evolución del mAP@0.5"
)})

wandb.log({"combined_map50-95": wandb.plot.line_series(
    xs=list(range(1, epochs_transfer + epochs_finetune + 1)),
    ys=[list(metrics_to_plot['map50-95']['transfer']) + list(metrics_to_plot['map50-95']['finetune'])],
    keys=["mAP@0.5:0.95"],
    title="Evolución del mAP@0.5:0.95"
)})

wandb.log({"combined_precision": wandb.plot.line_series(
    xs=list(range(1, epochs_transfer + epochs_finetune + 1)),
    ys=[list(metrics_to_plot['precision']['transfer']) + list(metrics_to_plot['precision']['finetune'])],
    keys=["Precisión"],
    title="Evolución de la Precisión"
)})

wandb.log({"combined_recall": wandb.plot.line_series(
    xs=list(range(1, epochs_transfer + epochs_finetune + 1)),
    ys=[list(metrics_to_plot['recall']['transfer']) + list(metrics_to_plot['recall']['finetune'])],
    keys=["Recall"],
    title="Evolución del Recall"
)})

In [ ]:
# ===================== ANÁLISIS DE FALSOS POSITIVOS Y FALSOS NEGATIVOS =====================
def analyze_error_cases(model, test_dir, test_labels_dir, conf_threshold=0.25, iou_threshold=0.5, max_samples=5):
    false_positives, false_negatives = [], []
    image_files = [f for f in os.listdir(test_dir) if f.endswith('.jpg')]
    np.random.shuffle(image_files)
    for img_file in tqdm(image_files, desc="Analizando errores"):
        img_path = os.path.join(test_dir, img_file)
        label_path = os.path.join(test_labels_dir, img_file.replace('.jpg', '.txt'))
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape
        gt_boxes = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    values = line.strip().split()
                    if len(values) == 5:
                        _, x, y, width, height = map(float, values)
                        x1 = int((x - width / 2) * w)
                        y1 = int((y - height / 2) * h)
                        x2 = int((x + width / 2) * w)
                        y2 = int((y + height / 2) * h)
                        gt_boxes.append({'box': [x1, y1, x2, y2], 'matched': False})
        with torch.no_grad():
            results = model.predict(img_path, conf=conf_threshold, verbose=False)[0]
        pred_boxes = []
        for box in results.boxes:
            xyxy = box.xyxy[0].cpu().numpy().astype(int)
            conf = float(box.conf[0].cpu().numpy())
            pred_boxes.append({'box': xyxy, 'conf': conf, 'matched': False})
        for gt_idx, gt in enumerate(gt_boxes):
            best_iou = 0
            best_match = None
            for pred_idx, pred in enumerate(pred_boxes):
                if pred['matched']:
                    continue
                box1 = gt['box']
                box2 = pred['box']
                x_left = max(box1[0], box2[0])
                y_top = max(box1[1], box2[1])
                x_right = min(box1[2], box2[2])
                y_bottom = min(box1[3], box2[3])
                if x_right < x_left or y_bottom < y_top:
                    continue
                intersection_area = (x_right - x_left) * (y_bottom - y_top)
                box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
                box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
                iou = intersection_area / float(box1_area + box2_area - intersection_area)
                if iou > best_iou and iou >= iou_threshold:
                    best_iou = iou
                    best_match = pred_idx
            if best_match is not None:
                gt_boxes[gt_idx]['matched'] = True
                pred_boxes[best_match]['matched'] = True
            else:
                if len(false_negatives) < max_samples:
                    fn_img = img.copy()
                    cv2.rectangle(fn_img, (gt['box'][0], gt['box'][1]), (gt['box'][2], gt['box'][3]), (255, 0, 0), 2)
                    false_negatives.append({'image': fn_img, 'filename': img_file, 'box': gt['box']})
        for pred in pred_boxes:
            if not pred['matched']:
                if len(false_positives) < max_samples:
                    fp_img = img.copy()
                    cv2.rectangle(fp_img, (pred['box'][0], pred['box'][1]), (pred['box'][2], pred['box'][3]), (0, 0, 255), 2)
                    false_positives.append({'image': fp_img, 'filename': img_file, 'box': pred['box'], 'confidence': pred['conf']})
        if len(false_positives) >= max_samples and len(false_negatives) >= max_samples:
            break
    return false_positives, false_negatives

print("\n=== ANÁLISIS DE FALSOS POSITIVOS Y FALSOS NEGATIVOS ===")
false_positives, false_negatives = analyze_error_cases(ft_model, images_test, labels_test)
if false_positives:
    plt.figure(figsize=(15, 5 * min(len(false_positives), 5)))
    for i, fp in enumerate(false_positives[:5]):
        plt.subplot(min(len(false_positives), 5), 1, i+1)
        plt.imshow(fp['image'])
        plt.title(f"Falso Positivo en {fp['filename']} (Confianza: {fp['confidence']:.2f})")
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"{RESULTS_PATH}/false_positives.png")
    plt.show()
else:
    print("No se encontraron falsos positivos en la muestra analizada.")
if false_negatives:
    plt.figure(figsize=(15, 5 * min(len(false_negatives), 5)))
    for i, fn in enumerate(false_negatives[:5]):
        plt.subplot(min(len(false_negatives), 5), 1, i+1)
        plt.imshow(fn['image'])
        plt.title(f"Falso Negativo en {fn['filename']}")
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"{RESULTS_PATH}/false_negatives.png")
    plt.show()
else:
    print("No se encontraron falsos negativos en la muestra analizada.")

wandb.log({"false_positives_count": len(false_positives), "false_negatives_count": len(false_negatives)})

In [ ]:
# ===================== COMPARACIÓN DE CONFIGURACIONES =====================
print("\n=== COMPARACIÓN DE CONFIGURACIONES ===")
transfer_df = pd.read_csv(f"{OUTPUT_PATH}/transfer_learning/results.csv")
finetune_df = pd.read_csv(f"{OUTPUT_PATH}/fine_tuning/results.csv")
config_summary = {
    'Transfer Learning': {
        'model': CONFIG['model_type'],
        'batch_size': CONFIG['batch_size'],
        'learning_rate': CONFIG['learning_rate_transfer'],
        'optimizer': 'SGD + Momentum',
        'epochs': CONFIG['epochs_transfer'],
        'final_map50': transfer_df['       metrics/mAP50(B)'].iloc[-1],
        'final_map50_95': transfer_df['    metrics/mAP50-95(B)'].iloc[-1],
        'final_precision': transfer_df['       metrics/precision(B)'].iloc[-1],
        'final_recall': transfer_df['          metrics/recall(B)'].iloc[-1]
    },
    'Fine-Tuning': {
        'model': CONFIG['model_type'],
        'batch_size': CONFIG['batch_size']//2,
        'learning_rate': CONFIG['learning_rate_finetune'],
        'optimizer': 'AdamW',
        'epochs': CONFIG['epochs_finetune'],
        'final_map50': finetune_df['       metrics/mAP50(B)'].iloc[-1],
        'final_map50_95': finetune_df['    metrics/mAP50-95(B)'].iloc[-1],
        'final_precision': finetune_df['       metrics/precision(B)'].iloc[-1],
        'final_recall': finetune_df['          metrics/recall(B)'].iloc[-1]
    }
}

print("\nComparación de configuraciones y resultados:")
df_config = pd.DataFrame(config_summary).T
print(df_config)

metrics_to_compare = ['final_map50', 'final_map50_95', 'final_precision', 'final_recall']
metrics_labels = {'final_map50': 'mAP@0.5', 'final_map50_95': 'mAP@0.5:0.95', 'final_precision': 'Precisión', 'final_recall': 'Recall'}
plt.figure(figsize=(12, 6))
x = np.arange(len(metrics_to_compare))
width = 0.35
transfer_values = [config_summary['Transfer Learning'][m] for m in metrics_to_compare]
finetune_values = [config_summary['Fine-Tuning'][m] for m in metrics_to_compare]
plt.bar(x - width/2, transfer_values, width, label='Transfer Learning')
plt.bar(x + width/2, finetune_values, width, label='Fine-Tuning')
plt.xlabel('Métrica')
plt.ylabel('Valor')
plt.title('Comparación de Métricas entre Transfer Learning y Fine-Tuning')
plt.xticks(x, [metrics_labels[m] for m in metrics_to_compare])
plt.legend()
plt.grid(True, alpha=0.3)
for i, v in enumerate(transfer_values):
    plt.text(i - width/2, v + 0.01, f'{v:.3f}', ha='center')
for i, v in enumerate(finetune_values):
    plt.text(i + width/2, v + 0.01, f'{v:.3f}', ha='center')
plt.tight_layout()
plt.savefig(f"{RESULTS_PATH}/metrics_comparison.png")
plt.show()

wandb.log({"config_comparison": wandb.Table(dataframe=df_config)})

In [ ]:
# ===================== ESTIMACIÓN DE LA CAPACIDAD DE GENERALIZACIÓN =====================
print("\n=== ESTIMACIÓN DE LA CAPACIDAD DE GENERALIZACIÓN ===")
def evaluate_generalization():
    val_results = ft_model.val(data=custom_yaml_path, split='val')
    test_results = ft_model.val(data=custom_yaml_path, split='test')
    val_metrics = val_results.results_dict
    test_metrics = test_results.results_dict
    metrics_to_compare = [
        ('metrics/mAP50(B)', 'mAP@0.5'),
        ('metrics/mAP50-95(B)', 'mAP@0.5:0.95'),
        ('metrics/precision(B)', 'Precisión'),
        ('metrics/recall(B)', 'Recall')
    ]
    differences, names, val_values, test_values = [], [], [], []
    for metric_key, metric_name in metrics_to_compare:
        val_value = val_metrics[metric_key]
        test_value = test_metrics[metric_key]
        rel_diff = (val_value - test_value) / (val_value + 1e-6) * 100
        differences.append(rel_diff)
        names.append(metric_name)
        val_values.append(val_value)
        test_values.append(test_value)
    return names, val_values, test_values, differences

try:
    metric_names, val_values, test_values, differences = evaluate_generalization()
    gen_data = {
        'Métrica': metric_names,
        'Validación': val_values,
        'Prueba': test_values,
        'Diferencia (%)': differences
    }
    gen_df = pd.DataFrame(gen_data)
    print("\nEvaluación de generalización:")
    print(gen_df)

    plt.figure(figsize=(10, 6))
    x = np.arange(len(metric_names))
    width = 0.35
    plt.bar(x - width/2, val_values, width, label='Validación')
    plt.bar(x + width/2, test_values, width, label='Prueba')
    plt.xlabel('Métrica')
    plt.ylabel('Valor')
    plt.title('Comparación entre Validación y Prueba')
    plt.xticks(x, metric_names)
    plt.legend()
    plt.grid(True, alpha=0.3)
    for i, v in enumerate(val_values):
        plt.text(i - width/2, v + 0.01, f'{v:.3f}', ha='center')
    for i, v in enumerate(test_values):
        plt.text(i + width/2, v + 0.01, f'{v:.3f}', ha='center')
    plt.tight_layout()
    plt.savefig(f"{RESULTS_PATH}/validation_test_comparison.png")
    plt.show()

    plt.figure(figsize=(10, 6))
    colors = ['green' if d < 5 else 'orange' if d < 10 else 'red' for d in np.abs(differences)]
    plt.bar(metric_names, np.abs(differences), color=colors)
    plt.axhline(y=5, color='r', linestyle='--', label='Umbral 5%')
    plt.xlabel('Métrica')
    plt.ylabel('Diferencia Absoluta (%)')
    plt.title('Gap entre Validación y Prueba')
    plt.grid(True, alpha=0.3)
    plt.legend()
    for i, v in enumerate(differences):
        plt.text(i, abs(v) + 0.5, f'{abs(v):.2f}%', ha='center')
    plt.tight_layout()
    plt.savefig(f"{RESULTS_PATH}/generalization_gap.png")
    plt.show()

    avg_diff = np.mean(np.abs(differences))
    print("\nInterpretación de la capacidad de generalización:")
    if avg_diff < 5:
        print("✅ EXCELENTE GENERALIZACIÓN: Diferencia promedio < 5%.")
    elif avg_diff < 10:
        print("⚠️ BUENA GENERALIZACIÓN: Diferencia promedio entre 5-10%.")
    else:
        print("❌ POSIBLE SOBREAJUSTE: Diferencia promedio > 10%.")
except Exception as e:
    print(f"No se pudo realizar la evaluación de generalización: {e}")

wandb.log({"generalization_metrics": wandb.Table(dataframe=gen_df)})

In [ ]:
# ===================== EXPORTACIÓN DEL MODELO FINAL =====================
print("\n=== EXPORTACIÓN DEL MODELO FINAL ===")

# Definir formatos de exportación (excluyendo 'pt')
formats = ['torchscript', 'onnx', 'openvino', 'tflite']
export_paths = {}
model_path = f"{OUTPUT_PATH}/fine_tuning/weights/best.pt"  # Ruta del modelo entrenado
ft_model = YOLO(model_path)

# Asegurar que la GPU se use si está disponible
device = 0 if torch.cuda.is_available() else 'cpu'
print(f"Usando dispositivo para exportación: {'GPU' if device == 0 else 'CPU'}")

for fmt in formats:
    try:
        # Definir la ruta esperada según el formato
        if fmt in ['torchscript', 'onnx']:
            expected_path = f"{OUTPUT_PATH}/fine_tuning/weights/best.{fmt}"
        elif fmt == 'openvino':
            expected_path = f"{OUTPUT_PATH}/fine_tuning/weights/best_openvino_model"
        elif fmt == 'tflite':
            expected_path = f"{OUTPUT_PATH}/fine_tuning/weights/best.tflite"
        else:
            expected_path = f"{OUTPUT_PATH}/final_model_{fmt}.{fmt}"

        # Exportar el modelo
        ft_model.export(
            format=fmt,
            imgsz=CONFIG['img_size'],
            device=device,
            optimize=False if fmt in ['onnx', 'openvino'] else True,
            dynamic=False,
            half=False,  # Evitar FP16 en exportación para compatibilidad
            workspace=4.0 if fmt == 'tflite' else None  # Optimización para TFLite
        )

        # Verificar que el archivo o directorio se creó
        if fmt == 'openvino':
            # OpenVINO crea un directorio
            if os.path.isdir(expected_path):
                export_paths[fmt] = expected_path
                print(f"Modelo exportado exitosamente en formato {fmt}: {expected_path}")
            else:
                print(f"Advertencia: No se encontró el directorio exportado para {fmt}")
        else:
            # Otros formatos crean un archivo
            if os.path.isfile(expected_path):
                export_paths[fmt] = expected_path
                print(f"Modelo exportado exitosamente en formato {fmt}: {expected_path}")
            else:
                print(f"Advertencia: No se encontró el archivo exportado para {fmt}")

    except Exception as e:
        print(f"Error al exportar en formato {fmt}: {str(e)}")
        export_paths[fmt] = None

# Loggear rutas de exportación en W&B
wandb.log({"exported_model_paths": export_paths})

# Verificar que al menos un formato se exportó correctamente
if not any(export_paths.values()):
    print("⚠️ Advertencia: No se exportó correctamente ningún formato del modelo.")
else:
    print("Exportación completada. Rutas de los modelos exportados:")
    for fmt, path in export_paths.items():
        if path:
            print(f"- {fmt}: {path}")

# Liberar memoria
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# ===================== INFORME FINAL =====================
print("\n=== INFORME FINAL ===")
report = f"""
REPORTE DE ENTRENAMIENTO - DETECCIÓN DE PERSONAS
Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Duración total del entrenamiento: {(time.time() - wandb.run.start_time) / 3600:.2f} horas

1. CONFIGURACIÓN
- Modelo: {CONFIG['model_type']}
- Tamaño de imagen: {CONFIG['img_size']}x{CONFIG['img_size']}
- Transfer Learning: {CONFIG['epochs_transfer']} épocas, LR={CONFIG['learning_rate_transfer']}, Batch={CONFIG['batch_size']}
- Fine-Tuning: {CONFIG['epochs_finetune']} épocas, LR={CONFIG['learning_rate_finetune']}, Batch={CONFIG['batch_size']//2}
- Dataset: {train_images_count} imágenes (train), {val_images_count} (val), {test_images_count} (test)

2. MÉTRICAS FINALES (TEST)
- Precisión: {val_metrics['metrics/precision(B)']:.4f}
- Recall: {val_metrics['metrics/recall(B)']:.4f}
- F1-Score: {f1_score:.4f}
- mAP@0.5: {val_metrics['metrics/mAP50(B)']:.4f}
- mAP@0.5:0.95: {val_metrics['metrics/mAP50-95(B)']:.4f}
- TPR: {tpr:.4f}
- FPR: {fpr:.4f}

3. ANÁLISIS DE ERRORES
- Falsos Positivos: {len(false_positives)}
- Falsos Negativos: {len(false_negatives)}

4. RENDIMIENTO POR TAMAÑO
{chr(10).join([f"- {size.capitalize()}: Precisión={metrics['precision']:.4f}, Recall={metrics['recall']:.4f}, F1={metrics['f1']:.4f}" for size, metrics in size_metrics.items()])}

5. VELOCIDAD DE INFERENCIA
{chr(10).join([f"- Batch {bs}: {metrics['fps']:.2f} FPS" for bs, metrics in speed_results.items()])}

6. MODELOS EXPORTADOS
{chr(10).join([f"- {fmt.upper()}: {path if path else 'No exportado'}" for fmt, path in export_paths.items()])}

7. CAPACIDAD DE GENERALIZACIÓN
- Diferencia promedio: {avg_diff:.2f}% {'(Excelente)' if avg_diff < 5 else '(Buena)' if avg_diff < 10 else '(Revisar posible sobreajuste)'}

Resultados y gráficos guardados en: {RESULTS_PATH}
Modelos guardados en: {OUTPUT_PATH}
"""

# Imprimir y guardar informe
print(report)
report_path = f"{RESULTS_PATH}/training_report.txt"
with open(report_path, 'w') as f:
    f.write(report)
print(f"Informe guardado en: {report_path}")

# Subir informe a W&B
wandb.save(report_path)

# ===================== LIMPIEZA Y FINALIZACIÓN =====================
print("\n=== FINALIZANDO ===")
# Liberar memoria
torch.cuda.empty_cache()
gc.collect()

# Cerrar W&B
wandb.finish()
print("Entrenamiento completado exitosamente. Todos los recursos han sido liberados.")

# Verificar espacio en Drive
import subprocess
space_output = subprocess.check_output(f"du -sh {OUTPUT_PATH}", shell=True).decode().split()[0]
print(f"Espacio utilizado en {OUTPUT_PATH}: {space_output}")